In [1]:
from torchvision import datasets
from PIL import Image

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import skimage as ski
import cv2
from tqdm import tqdm

import os

In [ ]:
# Load the data
md = "./.../Herb/dataset/DIMPSAR_org"

sv_dr = "./.../Herb/dataset/DIMPSAR_processed"

if not os.path.exists(sv_dr):
    os.makedirs(sv_dr)

In [3]:
for i in os.listdir(md):
    org_fldr = os.path.join(md, i)
    new_fldr = os.path.join(sv_dr, i)
    
    if not os.path.exists(new_fldr):
        os.makedirs(new_fldr)
        
    for img_file in tqdm(os.listdir(org_fldr)):
        # Load the image
        img_path = os.path.join(org_fldr, img_file)
        
        # Read the image
        img_org = mpimg.imread(img_path)
        
        # ------------------------------------------------
        # HSV COLOR SPACE
        # ----------------------
        # Convert the image to HSV color space
        hsv = cv2.cvtColor(img_org, cv2.COLOR_RGB2HSV)
        
        # Define range of green color in HSV
        lower_green = np.array([35, 40, 50])
        upper_green = np.array([85, 255, 255])
                
        # Threshold the HSV image to get only green colors
        mask = cv2.inRange(hsv, lower_green, upper_green)
        
        # ------------------------------------------------
        # DILATION
        # ----------------------
        # Image dilation (Puffing the image)
        mask_connect = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (2, 2))
        img_c = cv2.dilate(mask, mask_connect, iterations=2)
        
        # ------------------------------------------------
        # CONTOURS
        # ----------------------
        # Find contours
        contours, _ = cv2.findContours(img_c, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        contours = sorted(contours, key=cv2.contourArea, reverse=True)

        # Extract the largest contours
        contours = contours[:1]
        
        # ------------------------------------------------
        # COUNTOUR MASKING
        # ----------------------
        # Create Contour mask
        mask_con = np.zeros_like(img_org)
        cv2.drawContours(mask_con, contours, -1, (255, 255, 255), -1)
        
        # ------------------------------------------------
        # MASKING
        # ----------------------
        # Apply the mask to the original image
        img_masked = cv2.bitwise_and(img_org, mask_con)
        
        # ------------------------------------------------
        # WHITE BACKGROUND
        # ----------------------
        # Convert the image to a white background
        white_background = np.full_like(img_org, (255, 255, 255))
        
        # Place the leaf on the white background
        img_white = np.where(mask_con == 255, img_org, white_background)        
        
        # ------------------------------------------------
        # GAMMA CORRECTION
        # ----------------------
        # Apply gamma correction
        #img_gamma = ski.exposure.adjust_gamma(img_white, gamma=1)
        
        # ------------------------------------------------
        # CLAHE
        # ----------------------
        # Contrast Limited Adaptive Histogram Equalization
        # Adaptive histogram equalization
        #img_adapteq = ski.exposure.equalize_adapthist(img_gamma, clip_limit=0.001)
        
        # ------------------------------------------------
        # SAVE THE IMAGE
        # ----------------------
        sv_path = os.path.join(new_fldr, img_file)
        
        #plt.imsave(sv_path, img_white)
        
        # save using PIL | Higher quality images | * 255 (if adapt)
        img_sv = Image.fromarray((img_white).astype(np.uint8))
        img_sv.save(sv_path, format='JPEG', quality=100)

100%|██████████| 40/40 [00:00<00:00, 79.39it/s]
